In [1]:
import numpy as np 
import pandas as pd
import re 

import matplotlib.pyplot as plt
import seaborn as sns

import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem import SnowballStemmer

!pip install eli5
import eli5

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.random import sample_without_replacement

!pip install dtreeviz
from dtreeviz.trees import *

import graphviz

from string import punctuation

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import joblib

In [2]:
df = pd.read_csv('data_grupo13.csv', encoding='latin-1')
df

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,5440634,fbcd534ae53710ce39b8f0c64c1c6fb4708a502494d16b...,2018-03-08,0.824366,0.002982,0.016324,0.814987,0.002766,0.102328,0.009595,...,0.257196,0.002538,0.008044,0.005599,0.000434,0.001119,NaN,0.004633,0.004947,0.001014
1,722801,21930d38f66a0f7212cfa36ea0ba07a04db9afe627776c...,2017-12-04,0.424188,0.184747,0.041143,1.002571,0.004865,0.135947,0.006585,...,NaN,NaN,NaN,0.000314,0.001003,0.005322,NaN,0.009863,0.003724,0.002871
2,134854,064bc815588bc0c5629ef4cb3a9aa6eac3aa2a2318f5db...,2017-03-18,0.639963,0.006567,0.009535,0.818705,0.001143,0.128277,0.001888,...,NaN,NaN,NaN,0.003781,0.009854,0.001541,NaN,0.001492,0.001819,0.008832
3,3519306,a288b1a5994116ee8489ff8054759fa5b65804d0f8f341...,2018-03-06,0.437515,0.242892,0.223430,0.023620,0.001164,0.283376,0.007000,...,NaN,NaN,NaN,0.005641,0.002536,0.009831,NaN,0.000266,0.008466,0.001720
4,1117631,33a0617e54f8d17bd7e9e0a57875c802447522d0d601b9...,2017-11-13,0.934242,0.008750,0.009621,0.812051,0.005652,0.164320,0.006984,...,NaN,NaN,NaN,0.006492,0.007747,0.000889,NaN,0.000349,0.006727,0.008292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276567,5400079,f9e34a7fe4c79ac9c1b48bd509e76273cdebbaa64db079...,2017-06-17,0.547354,0.004800,0.447314,0.073896,0.008824,0.127696,0.009834,...,NaN,NaN,NaN,0.009388,0.001940,0.000094,NaN,0.004312,0.005863,0.009525
276568,3962870,b71c3dfd39a20399078e66c5b4a4c6bd923c42e2fdb2a0...,2017-08-16,0.976961,0.420968,0.024214,1.008787,0.005899,0.054443,0.003925,...,NaN,NaN,NaN,0.000497,0.001351,0.008252,NaN,0.008144,0.002436,0.003025
276569,612624,1c7fc3c8622587e27aae7cb61d6e88ad8058a947b0c9b7...,2017-09-23,0.346145,0.093374,1.004264,0.027010,0.006014,0.139173,0.007841,...,NaN,NaN,NaN,0.007268,0.004564,0.006531,NaN,0.001064,0.002482,0.001219
276570,4133771,bf1133692d356d677dac573ffeccb292a3d11b0c0c38a5...,2017-06-26,0.537823,0.001329,1.006889,0.023751,0.253425,0.168520,0.006843,...,NaN,NaN,NaN,0.007964,0.006620,0.001808,NaN,0.004633,0.004878,0.000593


In [11]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,fbcd534ae53710ce39b8f0c64c1c6fb4708a502494d16b...,2018-03-08,0.824366,0.002982,0.016324,0.814987,0.002766,0.102328,0.009595,0.010019,...,0.257196,0.002538,0.008044,0.005599,0.000434,0.001119,NaN,0.004633,0.004947,0.001014
1,21930d38f66a0f7212cfa36ea0ba07a04db9afe627776c...,2017-12-04,0.424188,0.184747,0.041143,1.002571,0.004865,0.135947,0.006585,0.008072,...,NaN,NaN,NaN,0.000314,0.001003,0.005322,NaN,0.009863,0.003724,0.002871
2,064bc815588bc0c5629ef4cb3a9aa6eac3aa2a2318f5db...,2017-03-18,0.639963,0.006567,0.009535,0.818705,0.001143,0.128277,0.001888,0.014288,...,NaN,NaN,NaN,0.003781,0.009854,0.001541,NaN,0.001492,0.001819,0.008832
3,a288b1a5994116ee8489ff8054759fa5b65804d0f8f341...,2018-03-06,0.437515,0.242892,0.223430,0.023620,0.001164,0.283376,0.007000,0.715525,...,NaN,NaN,NaN,0.005641,0.002536,0.009831,NaN,0.000266,0.008466,0.001720
4,33a0617e54f8d17bd7e9e0a57875c802447522d0d601b9...,2017-11-13,0.934242,0.008750,0.009621,0.812051,0.005652,0.164320,0.006984,0.003804,...,NaN,NaN,NaN,0.006492,0.007747,0.000889,NaN,0.000349,0.006727,0.008292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276567,f9e34a7fe4c79ac9c1b48bd509e76273cdebbaa64db079...,2017-06-17,0.547354,0.004800,0.447314,0.073896,0.008824,0.127696,0.009834,0.328632,...,NaN,NaN,NaN,0.009388,0.001940,0.000094,NaN,0.004312,0.005863,0.009525
276568,b71c3dfd39a20399078e66c5b4a4c6bd923c42e2fdb2a0...,2017-08-16,0.976961,0.420968,0.024214,1.008787,0.005899,0.054443,0.003925,0.006793,...,NaN,NaN,NaN,0.000497,0.001351,0.008252,NaN,0.008144,0.002436,0.003025
276569,1c7fc3c8622587e27aae7cb61d6e88ad8058a947b0c9b7...,2017-09-23,0.346145,0.093374,1.004264,0.027010,0.006014,0.139173,0.007841,1.099438,...,NaN,NaN,NaN,0.007268,0.004564,0.006531,NaN,0.001064,0.002482,0.001219
276570,bf1133692d356d677dac573ffeccb292a3d11b0c0c38a5...,2017-06-26,0.537823,0.001329,1.006889,0.023751,0.253425,0.168520,0.006843,1.132493,...,NaN,NaN,NaN,0.007964,0.006620,0.001808,NaN,0.004633,0.004878,0.000593
